In [5]:
import sys
import os
import xml.etree.ElementTree as ET

In [6]:
data_path = "data"
 
for id in ['test', 'train']:
#for id in ['train']:

    print id
    
    list_file_path = os.path.join(data_path,id+".txt")
    collection_path = os.path.join(data_path, id)
    print collection_path
    xml_files = !ls $collection_path/*.xml
    jpg_files = !ls $collection_path/*.jpg


    list_file = open(list_file_path, 'w')
    for f in xml_files:    
        root = ET.parse(f).getroot()

        class_id = root.find('ClassId').text

        image_filename = os.path.splitext(os.path.basename(f))[0] + ".jpg"
        asset_line = "%s %s\n" % (image_filename, class_id)
        list_file.write(asset_line)
    list_file.close()


train
data/train


import numpy as np
import caffe
import lmdb
from PIL import Image


lmdb_path = os.path.join(data_path,"test_data_lmdb")

ngray = 0

#in_db = lmdb.open(lmdb_path, map_size=int(1e12))
in_db = lmdb.open(lmdb_path, map_size=int(8*2**30))
with in_db.begin(write=True) as in_txn:
    for in_idx, in_ in enumerate(jpg_files):
        # load image:
        # - as np.uint8 {0, ..., 255}
        # - in BGR (switch from RGB)
        # - in Channel x Height x Width order (switch from H x W x C)
        im = np.array(Image.open(in_)) # or load whatever ndarray you need
        if len(im.shape) < 3:
            ngray += 1
            continue
        im = im[:,:,::-1]
        im = im.transpose((2,0,1))
        im_dat = caffe.io.array_to_datum(im)
        in_txn.put('{:0>10d}'.format(in_idx), im_dat.SerializeToString())
        if not in_idx % 100:
            print "%6i, ngrey=%i"%(in_idx, ngray)  
        if in_idx >= 3000:
            break
in_db.close()